In [1]:
import time
start_time = time.time()

# conda install -c conda-forge cassandra-driver
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from math import sin, cos, sqrt, atan2, radians, acos
import ast
from cassandra.cluster import Cluster
import warnings
warnings.filterwarnings("ignore")
yesterday = dt.date.today() - dt.timedelta(1)
t = (yesterday - dt.date(1970, 1, 1)).total_seconds()
imei=('911615400067990','861359036048302','861359036046330','869867030115077','869867030123980','869867030133294','869867030717930','869867030740296','869867030740858','869867030759999','869867030760005','869867030129649','869867030138533','869867030716288','869867030723334','869867030737730','869867030737995')
FMT = '%Y-%m-%d %H:%M:%S'

In [18]:
def execute_query(imei=[]): #Give input imei's in a list
    query = "select *from devicedata where imei In "+ str(imei) +" and devicetime>=" + str(int(t)) +" and devicetime<=" + str(int(t+86399))+"  ALLOW FILTERING;"
    print(str(imei))
    #print('Connecting to server...')
    cluster = Cluster(['45.114.143.13'])
    session = cluster.connect('assettl_tma')
    #print("--- %s seconds ---" % (time.time() - start_time))
    #print('Importing the data...')
    data = pd.DataFrame(list(session.execute(query)))
    session.shutdown()
    return(data)

In [22]:
# execute_query(imei)

data = pd.read_csv(r'C:\Users\INFO-DSK-04\Downloads\cassandra_data_20_11-2019.csv')

# Functions

In [23]:
data.columns

Index(['imei', 'devicetime', 'evc', 'evt', 'devicedata', 'packettype',
       'receivedtime', 'sensordata', 'sno', 'Latitude', 'Longitude', 'Speed',
       'TrackDateTime', 'distance', 'Angle', 'Sudden_Acceleration',
       'Harsh_Braking', 'Rash_Turning', 'Driver_scorecard'],
      dtype='object')

In [24]:
data=data[['imei', 'devicetime', 'devicedata']]

In [39]:
def preprocessing(data):
   # print('Data Preprocessing...')
    all_imei = data.imei.unique()
    td = []
    for i in range(0, len(data)):
        td.append(ast.literal_eval(data['devicedata'].iloc[i]))
    
    ldt = []
    for i in range(0, len(td)):
        ldt.append(td[i]['dvd'][0]['ldt'][0])
    
    sll = []
    for i in range(0, len(ldt)):
        sll.append({key: ldt[i][key] for key in ldt[0].keys() 
                               & {'lat', 'lon','spd'}})
    
    lat = []
    lon = []
    spd = []

    for l in sll:
        lat.append(l['lat'])
        lon.append(l['lon'])
        spd.append(l['spd'])
    
    data['Latitude'] = lat
    data['Longitude'] = lon
    data['Speed'] = spd

    data['Latitude'] = data['Latitude'].astype(float)
    data['Longitude'] = data['Longitude'].astype(float)
    data['Speed'] = data['Speed'].astype(int)

    dt = []
    for i in data.devicetime:
        dt.append(datetime.fromtimestamp(i))   
    data['TrackDateTime'] = dt



#    print("--- %s seconds ---" % (time.time() - start_time))

    data=data[['imei', 'devicetime', 'devicedata','Latitude', 'Longitude', 'Speed','TrackDateTime']]
    return(data)

In [40]:
#Function angle_between_vectors_degrees
def angle_between_vectors_degrees(u, v):
    """Return the angle between two vectors in any dimension space,
    in degrees."""
    return np.degrees(
        acos(np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))))

In [41]:
#func angle_between_vectors_degrees
def angle_between_vectors_degrees(u, v):
    """Return the angle between two vectors in any dimension space,
    in degrees."""
    return np.degrees(
        acos(np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))))

In [42]:
#Creating angle Column
def angle(df):
    angle1 = [0]
    for i in range(1,len(df),1):   
        if i != len(df['Latitude'])-1:
            # The points in tuple Latitude/Longitude degrees space
            A = [df.Latitude.iloc[i-1],df.Longitude.iloc[i-1]]
            B = [df.Latitude.iloc[i],df.Longitude.iloc[i]]
            C = [df.Latitude.iloc[i+1],df.Longitude.iloc[i+1]]
            try:
                if A == B == C:
                    angle1.append(0)

                elif ((A==B) | (B==C)):
                    angle1.append(0)

                else:
                    # Convert the points to numpy Latitudeitude/Longitudegitude radians space
                    a = np.radians(np.array(A))
                    b = np.radians(np.array(B))
                    c = np.radians(np.array(C))

                    # print(a,'\n',b,'\n',c,'\n\n')

                    # Vectors in Latitudeitude/Longitudegitude space
                    avec = a - b
                    cvec = c - b

                    # Adjust vectors for changed Longitudegitude scale at given Latitudeitude into 2D space
                    lat_ = b[0]
                    avec[1] *= cos(lat_)
                    cvec[1] *= cos(lat_)

                    # Find the angle between the vectors in 2D space
                    angle2deg = angle_between_vectors_degrees(avec, cvec)
                    angle1.append(angle2deg)

            except:
                angle1.append(0)
        else:
            angle1.append(0)
    return(angle1)
    #df['Angle'] = angle1
            

In [43]:
#Creating suuden_acc Column
def rapid_acc(df):
    sudden_acc= [0]
    for i in range(1, len(df), 1):    
        x = int((datetime.strptime(str(df.TrackDateTime.iloc[i]), FMT) - datetime.strptime(str(df.TrackDateTime.iloc[i-1]), FMT)).total_seconds())
        y = df.Speed.iloc[i] - df.Speed.iloc[i-1]

        if (((x > 0) & (x < 7)) & (df['distance'].iloc[i] < 0.025) & (y > 30)):
            sudden_acc.append(15)

        elif (((x > 0) & (x < 62)) & (df['distance'].iloc[i] < 0.03) & (y > 40)):
            sudden_acc.append(15)


        elif (((x > 0) & (x < 12)) & (df['distance'].iloc[i] < 0.04) & (y > 50)):
            sudden_acc.append(15)

        elif (((x > 0) & (x < 7) ) & (y > 30)):
            sudden_acc.append(15)

        else:
            sudden_acc.append(0)
    return(sudden_acc)

In [44]:
def distance(df):
    R = 6371 # Radius of the earth in km
    dist = [0]
    for i in range(1, len(df.Longitude), 1):
        dLat = radians(df.Latitude.iloc[i] - df.Latitude.iloc[i-1])
        dLon = radians(df.Longitude.iloc[i] - df.Longitude.iloc[i-1])
        rLat1 = radians(df.Latitude.iloc[i-1])
        rLat2 = radians(df.Latitude.iloc[i])
        a = sin(dLat/2) * sin(dLat/2) + cos(rLat1) * cos(rLat2) * sin(dLon/2) * sin(dLon/2) 
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        d = R * c
        dist.append(d)
    return(dist)
#     df['distance'] = dist

In [45]:
def harshbraking(df):
    harsh_braking = [0]
    for i in range(1, len(df), 1):    
        x = int((datetime.strptime(str(df.TrackDateTime.iloc[i]), FMT) - datetime.strptime(str(df.TrackDateTime.iloc[i-1]), FMT)).total_seconds())
        y = df.Speed.iloc[i] - df.Speed.iloc[i-1]

        if (((x > 0) & (x < 7)) & (df['distance'].iloc[i] < 0.012) & (y < -20)):
            harsh_braking.append(14)

        elif (((x > 0) & (x < 7)) & (df['distance'].iloc[i] < 0.027) & (y < -25)):
            harsh_braking.append(14)

        elif (((x > 0) & (x < 7)) & (y < -30)):
            harsh_braking.append(14)

        elif (((x > 0) & (x < 22)) & (df['distance'].iloc[i] < 0.032) & (y < -40)):
            harsh_braking.append(14)

        elif (((x > 0) & (x < 62)) & (df['distance'].iloc[i] < 0.027) & (y < -40)):
            harsh_braking.append(14)

        elif (((x > 0) & (x < 62)) & (df['distance'].iloc[i] < 0.04) & (y < -35) & (data.Speed.iloc[i-1] > 50)):
            harsh_braking.append(14)

        elif  ((df['distance'].iloc[i] < 0.012) & (y < -25)):
            harsh_braking.append(14)

        else:
            harsh_braking.append(0)
    return(harsh_braking)

In [46]:
def rashturn(df):
    rash_turn = [0]
    for i in range(1, len(df.Speed), 1):

        x = int((datetime.strptime(str(df.TrackDateTime.iloc[i]), FMT) - datetime.strptime(str(df.TrackDateTime.iloc[i-1]), FMT)).total_seconds())
        if ((x > 0) & (x < 6) & (df['distance'].iloc[i] < 0.02)):        

            if ((df['Angle'].iloc[i] > 30) & (df['Angle'].iloc[i] <= 50) & (df['Speed'].iloc[i] > 10)):            
                rash_turn.append(32)

            elif ((df['Angle'].iloc[i] > 50) & (df['Angle'].iloc[i] <= 90) & (df['Speed'].iloc[i] > 15)):            
                rash_turn.append(32)    


            elif ((df['Angle'].iloc[i] > 90) & (df['Angle'].iloc[i] <= 120) & (df['Speed'].iloc[i] > 18)):            
                rash_turn.append(32)

            elif ((df['Angle'].iloc[i] > 120) & (df['Angle'].iloc[i] <= 150) & (df['Speed'].iloc[i] > 25)):            
                rash_turn.append(32)

            elif ((df['Angle'].iloc[i] > 150) & (df['Angle'].iloc[i] <= 160) & (df['Speed'].iloc[i] > 30)):            
                rash_turn.append(32)


            elif ((df['Angle'].iloc[i] > 160) & (df['Angle'].iloc[i] <= 170) & (df['Speed'].iloc[i] > 35)):            
                rash_turn.append(32)

            else:
                rash_turn.append(0)

        else:
            rash_turn.append(0)
    return(rash_turn)

In [47]:
def driverscorecard(df):
    ds = []
    for i in range(0, len(df)):
        if i == 0:
            ds.append(100)

        elif ((df['Sudden_Acceleration'].iloc[i] == 15) & (df['Rash_Turning'].iloc[i] == 32)):
            ds.append(ds[i-1] - 10)

        elif ((df['Harsh_Braking'].iloc[i] == 14) & (df['Rash_Turning'].iloc[i] == 32)):
            ds.append(ds[i-1] - 8)

        elif ((df['Sudden_Acceleration'].iloc[i] == 15) | (df['Harsh_Braking'].iloc[i] == 14)):
            ds.append(ds[i-1] - 3)

        elif (df['Rash_Turning'].iloc[i] == 32):
            ds.append(ds[i-1] - 5)

        else:
            ds.append(ds[i-1])
    return(ds)
    

In [48]:
def remove_values_from_list(the_list, val):
      return [value for value in the_list if value != val]


## Generating Alerts...

In [49]:
def generatealerts(data):
    data=preprocessing(data)
    all_data = []
    all_imei = data.imei.unique()
    for i in all_imei:
        df = data[data['imei'] == i]

        df['distance'] = distance(df)   
        df['Angle'] = angle(df)     
        df['Sudden_Acceleration'] = rapid_acc(df)
        df['Harsh_Braking'] = harshbraking(df)
        df['Rash_Turning'] = rashturn(df)
    #changing format
        df['Sudden_Acceleration'] = df['Sudden_Acceleration'].astype(int)
        df['Harsh_Braking'] = df['Harsh_Braking'].astype(int)
        df['Rash_Turning'] = df['Rash_Turning'].astype(int)    

        df['Driver_scorecard'] = driverscorecard(df)

        all_data.append(df)

    All_Data = pd.concat(all_data)

    Alerts = All_Data[(All_Data['Sudden_Acceleration'] == 15) |
                              (All_Data['Harsh_Braking'] == 14) |
                              (All_Data['Rash_Turning'] == 32)]
    
    Alerts = Alerts[['imei', 'TrackDateTime','Latitude','Longitude','Sudden_Acceleration','Harsh_Braking','Rash_Turning','Driver_scorecard']]
    Alerts = Alerts.reset_index(drop=True)    
    Alerts['Alert_Type']= [(each[4],each[5],each[6]) for each in Alerts.values]
    Alerts['Alert_Type']=[remove_values_from_list(each, 0.0) for each in  Alerts.Alert_Type]
    return(Alerts)

In [50]:
def startprocess(data):
    data=preprocessing(data)
    Alerts=generatealerts(data)
    return(Alerts)

In [51]:
Alerts=startprocess(data)

In [52]:
Alerts

,imei,TrackDateTime,Latitude,Longitude,Sudden_Acceleration,Harsh_Braking,Rash_Turning,Driver_scorecard,Alert_Type
0,861359036046330,2019-11-20 09:57:45,17.411268,78.425087,0,14,0,97,[14]
1,861359036046330,2019-11-20 16:18:11,17.519048,78.385253,0,14,0,94,[14]
2,861359036046330,2019-11-20 16:18:51,17.519048,78.385253,0,14,0,91,[14]
3,861359036046330,2019-11-20 17:05:01,17.540747,78.393812,0,14,0,88,[14]
4,861359036046330,2019-11-21 01:18:55,17.303462,78.735700,0,14,0,85,[14]
5,861359036046330,2019-11-21 02:02:45,17.163127,79.297608,0,14,0,82,[14]
6,861359036046330,2019-11-21 03:45:35,16.795388,80.268512,0,14,0,79,[14]
